<a href="https://colab.research.google.com/github/liliya111darky/logical-ai/blob/notebooks/04_full_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install z3-solver transformers ipywidgets
!jupyter nbextension enable --py widgetsnbextension

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.7 MB/s eta 0:00:00
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [9]:
import sympy as sp
from sympy import sympify
from z3 import *
import sqlite3
from transformers import pipeline
import os
import json
import subprocess
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Подключаем NLP-модель
sentiment_pipeline = pipeline("sentiment-analysis")

# Подключаем/создаём базу фактов
FACTS_DB = "facts.db"
conn = sqlite3.connect(FACTS_DB)
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS facts (statement TEXT UNIQUE)")
conn.commit()

def save_fact(statement):
    try:
        cursor.execute("INSERT INTO facts VALUES (?)", (statement,))
        conn.commit()
    except sqlite3.IntegrityError:
        pass

def get_all_facts():
    cursor.execute("SELECT * FROM facts")
    return cursor.fetchall()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [10]:
def is_math_expression(statement):
    try:
        sympify(statement)
        return True
    except:
        return False

def check_math_truth(statement):
    try:
        expr = sympify(statement)
        if expr == True:
            return True
        elif expr == False:
            return False
        return bool(expr)
    except:
        return None

def check_logic_implication(premises, conclusion):
    solver = Solver()
    z3_premises = []
    for prem in premises:
        z3_premises.append(Bool(prem))
    z3_conclusion = Bool(conclusion)
    solver.add(And(*z3_premises) and Not(z3_conclusion))
    if solver.check() == unsat:
        return True
    else:
        return False

def analyze_input(user_input):
    # Математика
    if is_math_expression(user_input):
        truth = check_math_truth(user_input)
        if truth is True:
            save_fact(user_input)
            return {"type": "math_true", "message": f"✅ Математически верно: {user_input}"}
        elif truth is False:
            return {"type": "math_false", "message": f"❌ Математически неверно: {user_input}"}

    # Логика "Если ..., то ..."
    if "если" in user_input.lower() and "то" in user_input.lower():
        parts = user_input.lower().split("то")
        premise_part = parts[0].replace("если", "").strip()
        conclusion_part = parts[1].strip()
        premises = [p.strip() for p in premise_part.split("и")]
        if check_logic_implication(premises, conclusion_part):
            save_fact(user_input)
            return {"type": "valid_implication", "message": "✅ Логически верно"}
        else:
            return {"type": "invalid_implication", "message": "❌ Логически неверно"}

    # Проверка противоречий
    for fact in get_all_facts():
        if fact[0] == user_input:
            return {"type": "conflict", "message": "⚠️ Такой факт уже есть в базе"}

    # NLP-анализ
    sentiment = sentiment_pipeline(user_input)[0]
    return {"type": "text", "sentiment": sentiment}

In [11]:
# Файл истории
CHAT_HISTORY_FILE = "chat_history.json"

def load_chat_history():
    if os.path.exists(CHAT_HISTORY_FILE):
        with open(CHAT_HISTORY_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return []

def save_chat_history(history):
    with open(CHAT_HISTORY_FILE, "w", encoding="utf-8") as f:
        json.dump(history, f, ensure_ascii=False, indent=2)

# GitHub автосинхронизация
GITHUB_REPO_PATH = "/content/logical-ai"  # путь до твоего репозитория
def git_sync():
    try:
        subprocess.run(["cp", FACTS_DB, f"{GITHUB_REPO_PATH}/{FACTS_DB}"])
        subprocess.run(["cp", CHAT_HISTORY_FILE, f"{GITHUB_REPO_PATH}/{CHAT_HISTORY_FILE}"])
        subprocess.run(["git", "-C", GITHUB_REPO_PATH, "add", "."], check=True)
        subprocess.run(["git", "-C", GITHUB_REPO_PATH, "commit", "-m", "update memory"], check=True)
        subprocess.run(["git", "-C", GITHUB_REPO_PATH, "push"], check=True)
        print("✅ Синхронизировано с GitHub")
    except Exception as e:
        print("⚠️ Ошибка синхронизации:", e)

chat_history = load_chat_history()

In [13]:
input_box = widgets.Text(placeholder='Напиши сообщение ИИ...', description='Ты:')
send_button = widgets.Button(description="Отправить", button_style='success')
chat_output = widgets.Output()

def send_message(_):
    global chat_history
    user_text = input_box.value.strip()
    if not user_text:
        return

    chat_history.append(("Ты", user_text))

    result = analyze_input(user_text)
    if result["type"] == "math_true":
        ai_reply = result['message']
    elif result["type"] == "math_false":
        ai_reply = result['message']
    elif result["type"] == "valid_implication":
        ai_reply = result['message']
    elif result["type"] == "invalid_implication":
        ai_reply = result['message']
    elif result["type"] == "conflict":
        ai_reply = result['message']
    elif result["type"] == "text":
        sentiment = result["sentiment"]
        ai_reply = f"Это похоже на {sentiment['label']} (уверенность {sentiment['score']:.2f})"

    chat_history.append(("ИИ", ai_reply))
    save_chat_history(chat_history)
    git_sync()

    with chat_output:
        clear_output(wait=True)
        for speaker, text in chat_history:
            if speaker == "Ты":
                display(HTML(f"<b style='color:blue'>{speaker}:</b> {text}"))
            else:
                display(HTML(f"<b style='color:green'>{speaker}:</b> {text}"))
        display(HTML(f"<br><b>📚 Память фактов:</b> {[f[0] for f in get_all_facts()]}"))

    input_box.value = ""

send_button.on_click(send_message)
display(input_box, send_button, chat_output)

# Первичная отрисовка
with chat_output:
    clear_output(wait=True)
    for speaker, text in chat_history:
        if speaker == "Ты":
            display(HTML(f"<b style='color:blue'>{speaker}:</b> {text}"))
        else:
            display(HTML(f"<b style='color:green'>{speaker}:</b> {text}"))
    if chat_history:
        display(HTML(f"<br><b>📚 Память фактов:</b> {[f[0] for f in get_all_facts()]}"))

Text(value='', description='Ты:', placeholder='Напиши сообщение ИИ...')

Button(button_style='success', description='Отправить', style=ButtonStyle())

Output()